# Segmenting and Clustering Neighborhoods in Toronto

## Question 1

- The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
- Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
- More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
- If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.
- Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
- In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [2]:
import pandas as pd

Access to the html link by using pandas function (read_html)

In [3]:
html = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

data = pd.read_html(html, header=0)

There are four tables in the html page, then get the first table and save as df

In [4]:
df = data[0]
df

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


Drop the row if the column "Borough" contain "Not assigned"

In [5]:
df = df[df['Borough'] != 'Not assigned'].reset_index(drop=True)

Group by using 'Postcode' and 'Borough'. Combine the 'Neighbourhood'  if 'Postcode' and 'Borough' is duplicated 

In [6]:
df = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()

If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [8]:
for ind, row in df.iterrows():
    if row['Neighbourhood'] == 'Not assigned':
        row['Neighbourhood'] = row['Borough']
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Show the shape of the dataFrame

In [9]:
df.shape

(103, 3)

## Question 2
- Add Latitude and Longitude to the dataFrame

In [13]:
geo = pd.read_csv('Geospatial_Coordinates.csv')    
df1 = df.join(geo)
df1.head()

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


## Question 3
Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.
- to add enough Markdown cells to explain what you decided to do and to report any observations you make.
- to generate maps to visualize your neighborhoods and how they cluster together.

By using Folium, mark the location for each neighbourhood on the map

In [11]:
import folium
from geopy.geocoders import Nominatim

In [12]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))


map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df1['Latitude'], df1['Longitude'], df1['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

The geograpical coordinate of Toronto are 43.653963, -79.387207.


Using Foursquare API to gather information for North York Borough

Define Foursquare Credentials and Version

In [36]:
CLIENT_ID = 'KWANRTBXUUKNFLLOE5ESILFWPG1VQBE3DQF43PHZPF5MPTQS' # your Foursquare ID
CLIENT_SECRET = 'LXJ0XC3RHXM535KQCNFQC5QHTH1ECCECA0Z3CUVYE1AGK0PB' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

import requests

Your credentails:
CLIENT_ID: KWANRTBXUUKNFLLOE5ESILFWPG1VQBE3DQF43PHZPF5MPTQS
CLIENT_SECRET:LXJ0XC3RHXM535KQCNFQC5QHTH1ECCECA0Z3CUVYE1AGK0PB


Get the North York Borough information

In [37]:
North_York = df1[df1['Borough'] == 'North York'].reset_index()
North_York.head()

,index,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,17,M2H,North York,Hillcrest Village,M2H,43.803762,-79.363452
1,18,M2J,North York,"Fairview, Henry Farm, Oriole",M2J,43.778517,-79.346556
2,19,M2K,North York,Bayview Village,M2K,43.786947,-79.385975
3,20,M2L,North York,"Silver Hills, York Mills",M2L,43.757490,-79.374714
4,21,M2M,North York,"Newtonbrook, Willowdale",M2M,43.789053,-79.408493


## Explore Neighborhoods in North York

In [38]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Create a new dataframe with Neighbourhood name and Latitude and Longitude

In [39]:
LIMITED = 500
North_York_venues = getNearbyVenues(names=North_York['Neighbourhood'],
                                   latitudes=North_York['Latitude'],
                                   longitudes=North_York['Longitude'])

Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Bedford Park, Lawrence Manor East
Lawrence Heights, Lawrence Manor
Glencairn
Downsview, North Park, Upwood Park
Humber Summit
Emery, Humberlea


Show the size and the dataframe

In [40]:
print(North_York_venues.shape)
North_York_venues.head()

(201, 7)


,Neighbourhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Hillcrest Village,43.803762,-79.363452,Eagle's Nest Golf Club,43.805455,-79.364186,Golf Course
1,Hillcrest Village,43.803762,-79.363452,AY Jackson Pool,43.804515,-79.366138,Pool
2,Hillcrest Village,43.803762,-79.363452,Villa Madina,43.801685,-79.363938,Mediterranean Restaurant
3,Hillcrest Village,43.803762,-79.363452,Duncan Creek Park,43.805539,-79.360695,Dog Run
4,Hillcrest Village,43.803762,-79.363452,A.Y. Jackson Secondary School Track,43.805068,-79.366677,Athletics & Sports


The amount of venues near North York

In [42]:
North_York_venues.groupby('Neighbourhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Bathurst Manor, Downsview North, Wilson Heights",18,18,18,18,18,18
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",22,22,22,22,22,22
"CFB Toronto, Downsview East",3,3,3,3,3,3
Don Mills North,4,4,4,4,4,4
Downsview Central,4,4,4,4,4,4
Downsview Northwest,5,5,5,5,5,5
Downsview West,7,7,7,7,7,7
"Downsview, North Park, Upwood Park",4,4,4,4,4,4


## Analyze Each Neighbourhood

In [43]:
# one hot encoding
North_York_onehot = pd.get_dummies(North_York_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
North_York_onehot['Neighbourhood'] = North_York_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [North_York_onehot.columns[-1]] + list(North_York_onehot.columns[:-1])
North_York_onehot = North_York_onehot[fixed_columns]

North_York_onehot.head()

,Neighbourhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,...,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Store,Vietnamese Restaurant
0,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Hillcrest Village,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


The size of the dataframe

In [46]:
North_York_onehot.shape

(201, 94)

Group the Neighbourhood and calculate the mean

In [48]:
North_York_grouped = North_York_onehot.groupby('Neighbourhood').mean().reset_index()
North_York_grouped

,Neighbourhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,...,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Store,Vietnamese Restaurant
0,"Bathurst Manor, Downsview North, Wilson Heights",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.055556,0.000000,...,0.000000,0.000000,0.055556,0.055556,0.000000,0.000000,0.000000,0.000000,0.055556,0.000000
1,Bayview Village,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.250000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Bedford Park, Lawrence Manor East",0.000000,0.000000,0.045455,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.090909,0.000000,0.045455,0.000000,0.000000,0.000000,0.000000
3,"CFB Toronto, Downsview East",0.000000,0.333333,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Don Mills North,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Downsview Central,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Downsview Northwest,0.000000,0.000000,0.000000,0.000000,0.000000,0.2,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Downsview West,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.142857,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,"Downsview, North Park, Upwood Park",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.250000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,"Emery, Humberlea",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [49]:
North_York_grouped.shape

(22, 94)

Find the Top 5 venues within the Neighbourhood

In [50]:
num_top_venues = 5

for hood in North_York_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = North_York_grouped[North_York_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor, Downsview North, Wilson Heights----
                       venue  freq
0                Coffee Shop  0.11
1                Pizza Place  0.06
2              Shopping Mall  0.06
3                      Diner  0.06
4  Middle Eastern Restaurant  0.06


----Bayview Village----
                 venue  freq
0  Japanese Restaurant  0.25
1   Chinese Restaurant  0.25
2                 Bank  0.25
3                 Café  0.25
4            Juice Bar  0.00


----Bedford Park, Lawrence Manor East----
                  venue  freq
0           Coffee Shop  0.09
1      Sushi Restaurant  0.09
2    Italian Restaurant  0.09
3         Grocery Store  0.05
4  Fast Food Restaurant  0.05


----CFB Toronto, Downsview East----
                  venue  freq
0               Airport  0.33
1     Electronics Store  0.33
2                  Park  0.33
3                   Gym  0.00
4  Gym / Fitness Center  0.00


----Don Mills North----
                  venue  freq
0  Gym / Fitness Center  0.25
1  Car

Create the new dataframe and display the top 10 venues for each neighborhood.

In [51]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [54]:
import numpy as np

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']

for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = North_York_grouped['Neighbourhood']

for ind in np.arange(North_York_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(North_York_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Downsview North, Wilson Heights",Coffee Shop,Bank,Middle Eastern Restaurant,Deli / Bodega,Pharmacy,Pizza Place,Bridal Shop,Diner,Sandwich Place,Shopping Mall
1,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Vietnamese Restaurant,Discount Store,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
2,"Bedford Park, Lawrence Manor East",Coffee Shop,Italian Restaurant,Sushi Restaurant,Café,Grocery Store,Comfort Food Restaurant,Indian Restaurant,Fast Food Restaurant,Butcher,Pharmacy
3,"CFB Toronto, Downsview East",Airport,Electronics Store,Park,Vietnamese Restaurant,Diner,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop
4,Don Mills North,Caribbean Restaurant,Café,Gym / Fitness Center,Japanese Restaurant,Vietnamese Restaurant,Diner,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega


## Cluster Neighborhoods

In [56]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

North_York_grouped_clustering = North_York_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(North_York_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 4, 0, 3, 0, 0, 0, 1])

Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [57]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

North_York_merged = North_York

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
North_York_merged = North_York_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

North_York_merged.head()

,index,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,17,M2H,North York,Hillcrest Village,M2H,43.803762,-79.363452,0.0,Dog Run,Golf Course,Mediterranean Restaurant,Athletics & Sports,Pool,Diner,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop
1,18,M2J,North York,"Fairview, Henry Farm, Oriole",M2J,43.778517,-79.346556,0.0,Coffee Shop,Clothing Store,Salon / Barbershop,Japanese Restaurant,Juice Bar,Liquor Store,Candy Store,Fast Food Restaurant,Movie Theater,Cosmetics Shop
2,19,M2K,North York,Bayview Village,M2K,43.786947,-79.385975,0.0,Chinese Restaurant,Café,Bank,Japanese Restaurant,Vietnamese Restaurant,Discount Store,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
3,20,M2L,North York,"Silver Hills, York Mills",M2L,43.757490,-79.374714,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,21,M2M,North York,"Newtonbrook, Willowdale",M2M,43.789053,-79.408493,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(North_York_merged['Latitude'], North_York_merged['Longitude'], North_York_merged['Neighbourhood'], North_York_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='rainbow[cluster-1]',
        fill=True,
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters